# Nodal price

## Data


In [1]:
# Import Data

## Conventional Generators data
Conventional_generators = [
        1 2 7 13 15 15 16 18 21 22 23 23;
        13.32 13.32 20.7 20.93 26.11 10.52 10.52 6.02 5.47 7 10.52 10.89;
        1.68 1.68 3.30 4.07 1.89 5.48 5.48 4.98 5.53 8.00 3.45 5.11;
        2.32 2.32 4.67 3.93 3.11 3.52 3.52 5.02 4.97 6.00 2.52 2.89;
        106.4 106.4 245 413.7 42 108.5 108.5 280 280 210 217 245;
        48 48 84 216 42 36 36 60 60 48 72 48;
        48 48 84 216 42 36 36 60 60 48 72 48;
    ]
##  Wind Farms data

Wind_farms = [
    3 5 16 21;
    500 500 300 300;
    120.54 115.52 53.34 38.16;
]

## Demands
Demands = [
    1 2 3 4 5 6 7 8 9 10 13 14 15 16 18 19 20;
    84 75 139 58 55 106 97 132 135 150 205 150 245 77 258 141 100;
    500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500 500;
    ]

## Transmission_lines Data
Transmission_fir = [
    1 1 1 2 2 3 3 4 5 6 7 8 8 9 9 10 10;
    2 3 5 4 6 9 24 9 10 10 8 9 10 11 12 11 12;
    0.0146 0.2253 0.0907 0.1356 0.205 0.1271 0.084 0.111 0.094 0.0642 0.0652 0.1762 0.1762 0.084 0.084 0.084 0.084;
    175 175 350 175 175 175 400 175 350 175 350 175 175 400 400 400 400;
]

Transmission_sec =[
    11 11 12 12 13 14 15 15 15 16 16 17 17 18 19 20 21;
    13 14 13 23 23 16 16 21 24 17 19 18 22 21 20 23 22;
    0.0488 0.0426 0.0488 0.0985 0.0884 0.0594 0.0172 0.0249 0.0529 0.0263 0.0234 0.0143 0.1069 0.0132 0.0203 0.0112 0.0692;
    500 500 500 500 250 250 500 400 500 500 500 500 500 1000 1000 1000 500;
]

Transmission_lines = hcat(Transmission_fir, Transmission_sec)

# Get parameters from Data
## number of Conventional_generators
N_G = size(Conventional_generators,2)
## number of Wind_farms
N_W = size(Wind_farms,2)
## Number of Demands
N_D = size(Demands,2)
## Number of Transmission_lines
N_l = size(Transmission_lines,2)
## Number of nodes
(N_n, ) = findmax(Transmission_lines[1:2,:])
N_n = Int64.(N_n)
## Number of zones
N_a = 2 

## Susceptance of lines
Susceptance = 1 ./ Transmission_lines[3,:]

U = (Demands[2,:] + ones(N_D) *45)'

C = (Conventional_generators[5,1:end]/2)'

T = zeros(4)'

P_cg_max = Conventional_generators[5,1:end]
P_cg_min = zeros(N_G)

P_wt_max = Wind_farms[3,:]
P_wt_min = zeros(N_W)

P_dem_max = Demands[2,:]
P_dem_min = zeros(N_D);

## Functions

In [2]:
function swaprow!(x,i,j)
         for k in axes(x,2)
           idata = x[i,k]
           x[i,k] = x[j,k]
           x[j,k] = idata
         end
       end

function getbuscxn(Transmission_lines,N_n)
  X = Transmission_lines[1:2,:]
    swaprow!(X,1,2)
    Ωₙ = Vector{Vector{Int8}}()
    Ω_idx =  Vector{Vector{Int8}}()
        for i in 1:N_n
                x = Transmission_lines[1:2,:]
                a = findall(x -> x==i, x)
                b = getindex.(a, 2)
                push!(Ωₙ,X[a])
                push!(Ω_idx,b)
        end
        return Ωₙ, Ω_idx
end

function getelmcxt(Demands, Conventional_generators, Wind_farms, N_n)

    d = Vector{Vector{Int8}}()
    y1 = copy(Demands[1,:])
            for i in 1:N_n
                a = findall(y1 -> y1==i, y1)
                push!(d,a)
            end

    g = Vector{Vector{Int8}}()
    y2 = copy(Conventional_generators[1,:])
            for i in 1:N_n
                a = findall(y2 -> y2==i, y2)
                push!(g,a)
            end

    w = Vector{Vector{Int8}}()
    y = copy(Wind_farms[1,:])
            for i in 1:N_n
                a = findall(y -> y==i, y)
                push!(w,a)
            end

return d,g,w
end

function getelmzone(Demands, Conventional_generators, Wind_farms, N_n)

  (d,g,w) = getelmcxt(Demands, Conventional_generators, Wind_farms, N_n)
    
    zone_A =[1,2,3,4,5,6,7,8,9,10,11,12,13]
    zone_B =[14,15,16,17,18,19,20,21,22,23,24]

    d1 = copy.(d)
    a = []
    for i in zone_A
        a = vcat(a,d1[i])
    end
    d2 = copy.(d)
    b = []
    for i in zone_B
        b = vcat(b,d2[i])
    end
    d_z =[[a];[b]]
    
    g1 = copy.(g)
    a = []
    for i in zone_A
        a = vcat(a,g1[i])
    end
    g2 = copy.(g)
    b = []
    for i in zone_B
        b = vcat(b,g2[i])
    end
    g_z =[[a];[b]]
    
    w1 = copy.(w)
    a = []
    for i in zone_A
        a = vcat(a,w1[i])
    end
    w2 = copy.(w)
    b = []
    for i in zone_B
        b = vcat(b,w2[i])
    end
    w_z =[[a];[b]]
    
return d_z, g_z, w_z
end

getelmzone (generic function with 1 method)

## Inizio script

In [3]:
(d,g,w) = getelmcxt(Demands, Conventional_generators, Wind_farms, N_n) # Ψₙ 
(Ωₙ,Ω_idx) = getbuscxn(Transmission_lines,N_n); # Ωₙ
(d_z, g_z, w_z) = getelmzone(Demands, Conventional_generators, Wind_farms, N_n) # Ψₐ
Ωₐ = [[2],[1]] # Ωₐ

2-element Vector{Vector{Int64}}:
 [2]
 [1]

In [4]:
## calculate the value of ATC (Available Transfer Capacities) between zones

N_a = 2
zone_A =[1,2,3,4,5,6,7,8,9,10,11,12,13]
zone_B =[14,15,16,17,18,19,20,21,22,23,24]
ATC = zeros(N_a,N_a)
cxn_line = [7,18,21,22]
ATC[1,2] = sum(Transmission_lines[4,cxn_line])
ATC[2,1] = ATC[1,2]

1650.0

In [6]:
using JuMP
using GLPK


vector_model = Model(GLPK.Optimizer)

@variable(vector_model, P_G[1:N_G] >= 0)
@variable(vector_model, P_W[1:N_W] >= 0)
@variable(vector_model, P_D[1:N_D] >= 0)
@variable(vector_model, f_ab[1:N_a,1:N_a])

@constraint(vector_model, cg_con[i=1:N_G], P_cg_min[i] <= P_G[i] <= P_cg_max[i])
@constraint(vector_model, wt_con[i=1:N_W], P_wt_min[i] <= P_W[i] <= P_wt_max[i])
@constraint(vector_model, dem_con[i=1:N_D], P_dem_min[i] <= P_D[i] <= P_dem_max[i]);


@constraint(vector_model, bal_con[i=1:N_a], sum(P_D[d_z[i]]) + sum(f_ab[i,:]) - sum(P_G[g_z[i]]) - sum(P_W[w_z[i]]) .== 0)
@constraint(vector_model, trans[i=1:N_a,j=1:N_a], f_ab[i,j] == -f_ab[j,i])

@constraint(vector_model, flow_trans[i=1:N_a,j=1:N_a], - ATC[i,j] <= f_ab[i,j] <= ATC[i,j])

@objective(vector_model, Max, - T*P_W - C*P_G + U*P_D)

#print(vector_model)
optimize!(vector_model)
@show termination_status(vector_model)
@show primal_status(vector_model)
@show dual_status(vector_model)
@show objective_value(vector_model)
@show value.(P_G)
@show value.(P_W)
@show value.(P_D)
@show value.(f_ab);

termination_status(vector_model) = MathOptInterface.OPTIMAL
primal_status(vector_model) = MathOptInterface.FEASIBLE_POINT
dual_status(vector_model) = MathOptInterface.FEASIBLE_POINT
objective_value(vector_model) = 253295.69
value.(P_G) = [106.4, 106.4, 245.0, 0.0, 42.0, 108.5, 108.5, 141.64000000000004, 0.0, 210.0, 217.0, 245.0]
value.(P_W) = [120.54, 115.52, 53.34, 38.16]
value.(P_D) = [0.0, 0.0, 139.0, 0.0, 0.0, 106.0, 97.0, 132.0, 135.0, 150.0, 205.0, 150.0, 245.0, 0.0, 258.00000000000006, 141.0, 100.0]
value.(f_ab) = [0.0 -270.14; 270.14000000000004 0.0]


# Marcket Clearing Price

In [8]:
## Market Clearing Price
λₐ = Vector{Float64}(undef,N_a)
for i = 1:N_a
    λₐ[i] = shadow_price(bal_con[i])
end
λₐ

2-element Vector{Float64}:
 140.0
 140.0

# Please vary the value of ATC,
and find at which value of ATC, thesetwo prices become equal (or unequal).

In [28]:
ATC[1,2] = 270.0
ATC[2,1] = ATC[1,2]
vector_model = Model(GLPK.Optimizer)

@variable(vector_model, P_G[1:N_G] >= 0)
@variable(vector_model, P_W[1:N_W] >= 0)
@variable(vector_model, P_D[1:N_D] >= 0)
@variable(vector_model, f_ab[1:N_a,1:N_a])

@constraint(vector_model, cg_con[i=1:N_G], P_cg_min[i] <= P_G[i] <= P_cg_max[i])
@constraint(vector_model, wt_con[i=1:N_W], P_wt_min[i] <= P_W[i] <= P_wt_max[i])
@constraint(vector_model, dem_con[i=1:N_D], P_dem_min[i] <= P_D[i] <= P_dem_max[i]);

@constraint(vector_model, bal_con[i=1:N_a], sum(P_D[d_z[i]]) + sum(f_ab[i,:]) - sum(P_G[g_z[i]]) - sum(P_W[w_z[i]]) .== 0)
@constraint(vector_model, trans[i=1:N_a,j=1:N_a], f_ab[i,j] == -f_ab[j,i])

@constraint(vector_model, flow_trans[i=1:N_a,j=1:N_a], - ATC[i,j] <= f_ab[i,j] <= ATC[i,j])

@objective(vector_model, Max, - T*P_W - C*P_G + U*P_D)

#print(vector_model)
optimize!(vector_model)
@show termination_status(vector_model)
@show primal_status(vector_model)
@show dual_status(vector_model)
@show objective_value(vector_model)
@show value.(P_G)
@show value.(P_W)
@show value.(P_D)
@show value.(f_ab);

termination_status(vector_model) = MathOptInterface.OPTIMAL
primal_status(vector_model) = MathOptInterface.FEASIBLE_POINT
dual_status(vector_model) = MathOptInterface.FEASIBLE_POINT
objective_value(vector_model) = 253295.41
value.(P_G) = [106.4, 106.4, 245.0, 0.0, 42.0, 108.5, 108.5, 141.5, 0.0, 210.0, 217.0, 245.0]
value.(P_W) = [120.54, 115.52, 53.34, 38.16]
value.(P_D) = [0.0, 0.0, 139.0, 0.0, 0.0, 106.0, 96.85999999999997, 132.0, 135.0, 150.0, 205.0, 150.0, 245.0, 0.0, 258.0, 141.0, 100.0]
value.(f_ab) = [0.0 -270.0; 270.0 0.0]


In [26]:
## Market Clearing Price
λₐ = Vector{Float64}(undef,N_a)
for i = 1:N_a
    λₐ[i] = shadow_price(bal_con[i])
end
λₐ

2-element Vector{Float64}:
 142.0
 140.0